In [2]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv("../first.csv")

# extract style from style & color code
df['STYLE'] = df['UNIV_STYLE_COLOR_CD'].str.split('-', expand=True)[0]
df['TRANS_DT']= pd.to_datetime(df['TRANS_DT'])

# what shape do we want the data in?
# one row per timeseries
# columns: id, timestamp, target 
# timestamp and target should be same length
# shapes: int, list of datetimes, list of floats
df.drop(columns=['ORD_KEY', 'BOW_DATE', 'GTIN', 'UNIV_STYLE_COLOR_CD',
       'UNIV_SZ_CD', 'CHANNEL', 'COUNTRY', 'GEO', 'ZIP_CD', 'PLANT_CD',
       'RETURN_UNITS', 'NET_SLS_UNITS', 'CLEARANCE_IND',
       'GROSS_AMT_USD', 'GROSS_AMT_LC', 'NET_SLS_AMT_USD', 'NET_SLS_AMT_LC',
       'RETURN_AMT_LC', 'RETURN_AMT_USD', 'MSRP_LC', 'MSRP_USD'], inplace=True)
df = df.groupby(['STYLE', 'TRANS_DT'])['GROSS_UNITS'].sum().reset_index()
df.head()

,STYLE,TRANS_DT,GROSS_UNITS
0,408452,2021-08-08 00:00:00+00:00,1
1,408452,2022-06-25 00:00:00+00:00,1
2,408452,2022-07-23 00:00:00+00:00,1
3,415445,2020-08-28 00:00:00+00:00,1
4,415445,2021-05-25 00:00:00+00:00,1


In [4]:
# Create a complete date range
dt_min = df['TRANS_DT'].min()
dt_max = df['TRANS_DT'].max()
all_dates = pd.date_range(start=dt_min, end=dt_max, freq='D')
all_styles = df['STYLE'].unique()

# Create a dataframe with every combination of style and date
complete_index = pd.MultiIndex.from_product([all_styles, all_dates], names=['STYLE', 'TRANS_DT'])
df_complete = pd.DataFrame(index=complete_index).reset_index()
df_complete.head()

,STYLE,TRANS_DT
0,408452,2020-07-21 00:00:00+00:00
1,408452,2020-07-22 00:00:00+00:00
2,408452,2020-07-23 00:00:00+00:00
3,408452,2020-07-24 00:00:00+00:00
4,408452,2020-07-25 00:00:00+00:00


In [5]:
# Merge with the original dataframe to fill in the missing combinations with 0s
df = pd.merge(df_complete, df, on=['STYLE', 'TRANS_DT'], how='left').fillna(0)
df.head()

,STYLE,TRANS_DT,GROSS_UNITS
0,408452,2020-07-21 00:00:00+00:00,0.0
1,408452,2020-07-22 00:00:00+00:00,0.0
2,408452,2020-07-23 00:00:00+00:00,0.0
3,408452,2020-07-24 00:00:00+00:00,0.0
4,408452,2020-07-25 00:00:00+00:00,0.0


In [7]:
# Group by style and create lists of timestamps and gross units
grouped_df = df.groupby('STYLE').agg({'TRANS_DT': lambda x: list(x), 'GROSS_UNITS': lambda x: list(x)}).reset_index()

# Rename columns for clarity
grouped_df.rename(columns={'STYLE': 'id', 'TRANS_DT': 'timestamp', 'GROSS_UNITS': 'target'}, inplace=True)
grouped_df.head()

,id,timestamp,target
0,408452,"[2020-07-21 00:00:00+00:00, 2020-07-22 00:00:0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,415445,"[2020-07-21 00:00:00+00:00, 2020-07-22 00:00:0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,505835,"[2020-07-21 00:00:00+00:00, 2020-07-22 00:00:0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,553558,"[2020-07-21 00:00:00+00:00, 2020-07-22 00:00:0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,554724,"[2020-07-21 00:00:00+00:00, 2020-07-22 00:00:0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [8]:
# Validate that the targets are not all 0
[sum(x) for x in grouped_df['target']]

[3.0,
 16.0,
 1.0,
 6.0,
 1.0,
 3.0,
 22.0,
 1.0,
 1.0,
 12.0,
 1.0,
 4.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 3.0,
 1.0,
 1.0,
 10.0,
 1.0,
 1.0,
 1.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 11.0,
 1.0,
 1.0,
 2.0,
 7.0,
 1.0,
 1.0,
 1.0,
 30.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 3.0,
 1.0,
 1.0,
 3.0,
 1.0,
 1.0,
 13.0,
 1.0,
 1.0,
 1.0,
 1.0,
 6.0,
 1.0,
 2.0,
 4.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 17.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 4.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 4.0,
 1.0,
 1.0,
 4.0,
 1.0,
 2.0,
 1.0,
 1.0,
 5.0,
 4.0,
 3.0,
 1.0,
 1.0,
 6.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 10.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 4.0,
 10.0,
 2.0,
 1.0,
 1.0,
 24.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 3.0,
 1.0,

In [ ]:
# Save the dataframe to a csv file and parquet file
grouped_df.to_csv('dtc_timeseries.csv', index=False)
grouped_df.to_parquet('dtc_timeseries.parquet', index=False)